# COVID-19 Detection Using FHE

## Introduction
This is a neural network demo that shows how to perform COVID-19 predictions using encrypted Computerized Tomography (CT) scans with less than 32GB of RAM in about a minute. We use a neural network architecture of 3 convolutional layers and 2 fully connected layers that supports a large input image size of 224x224x3. As you can see, there isn't a lot of code required to perform this in a fully encrypted fashion.

## Use case
A potential use case in the healthcare domain is predictive analysis on encrypted medical data using a cloud service. A hospital’s data center is unlikely to match the scalability and efficiency of a cloud service and due to privacy risks and healthcare regulations, it is often impractical for hospitals to make the transition to cloud. FHE can help improve the acceptance of data-sharing protocols with third parties by outsourcing computations in a fully encrypted fashion and can help accelerate learning from real-world data. 

By following the image below, you can see that a *trusted* hospital (privileged client) can encrypt a CT image into unreadable data called ciphertext **(Step 1)** and send the encrypted CT image to an unprivileged server in a *less trusted* cloud environment. In **Step 2**, encrypted classification can be performed in the cloud environment to determine if a patient has COVID-19 given a data set containing encrypted CT images sent by the hospital. It is important to note that the on the cloud server, CT data and the results are always encrypted and therefore confidential and unreadable by the cloud server during this process. The encrypted health prediction is sent back to the hospital to decrypt the predictions **(Step 3)**; the hospital is the only entity that has access to the private key and has the privilege to decrypt the results.

**NOTE: while the client and server are not literally separated (nor demonstrating true remote cloud computation), the concepts generalize. One can imagine running the trusted code on local environment and the prediction code on a less trusted environment like the cloud. Additionally, we are working on an FHE cloud service that simplifies this process.**

<br>

<p align="center">
<img src="img/DemoCovid/Slide1.PNG"  width="1000" height="500" align="center"/>
</p>

<br>


<br>

## Step 0. Initialization

The first thing we need to do when working with pyhelayers is to import the library and  initialize the different data objects. Then create an instance of a Context which is the main HElayers object to be able to work with encrypted data. We initialize the context object with a settings profile that helps to define the various configuration settings that are needed to run. 

In [ ]:
import numpy as np
import os
import psutil
import utils
import pyhelayers
import utils 

utils.verify_memory(min_memory_size=25)

input_dir = utils.get_data_sets_dir() + "/covid_inference/" # for the model



In [ ]:
#### Load the NN model weights
# The neural network consists of 3 convolutional layers and 2 fully connected layers

hyper_params = pyhelayers.PlainModelHyperParams()
nnp = pyhelayers.NeuralNetPlain()
nnp.init_from_files(hyper_params, [input_dir + "model.json", input_dir + "model.h5"])
print("Loaded plain model")

# Labels of inference results: 
labels = ['Pneumonia','Healthy','COVID19']

#### Automatic optimization
# For the given model, we optimize the underlying FHE library's parameter, and various parameters related to the inference algorithm.

he_run_req = pyhelayers.HeRunRequirements()
he_run_req.set_he_context_options([pyhelayers.DefaultContext()])
he_run_req.set_model_encrypted(False)
profile = pyhelayers.HeModel.compile(nnp, he_run_req)

batch_size = profile.get_optimal_batch_size()

#### Context setup
# Here we initialize the library using parameters chosen automatically in the optimization process of the previous step

context = pyhelayers.HeModel.create_context(profile)
print('HE Context ready')

process = psutil.Process(os.getpid())
print("Memory consumption (GB): ", process.memory_info().rss / (1000*1000*1000))

nn = pyhelayers.NeuralNet(context)
nn.encode(nnp, profile)

<br>

## Step 1. Hospital encrypts the CT image into a ciphertext
Encryption of the CT image is performed in the trusted and privileged client environment

In [ ]:
import h5py
with h5py.File(input_dir + "covidCT_test_image_covid.h5", 'r') as f:
    data = f['covidCT_test_images'][:]
    data = np.expand_dims(np.transpose(data,[1,2,0]),axis=0)
    
plain_samples = data

#### Encrypt the CT image of healthy patient

# %%time

iop = nn.create_io_processor()
samples = pyhelayers.EncryptedData(context)
iop.encode_encrypt_inputs_for_predict(samples, [plain_samples])
print('Encrypted samples')

process = psutil.Process(os.getpid())
print("Memory consumption (GB): ", process.memory_info().rss / (1000*1000*1000))

<br>

## Step 2. Homomorphic inference in the cloud for detection of COVID-19
We can now run the inference on the encrypted CT data to obtain encrypted results. This computation uses a public key to manipulate completely encrypted values.

**NOTE: the data and the results always remain encrypted and are kept unreadable and confidential from the cloud server performing the computation.**

In [ ]:
%%time

utils.start_timer()
predictions = pyhelayers.EncryptedData(context)
nn.predict(predictions, samples)
duration=utils.end_timer('predict')
utils.report_duration('predict per sample',duration/batch_size)

process = psutil.Process(os.getpid())
print("Memory consumption (GB): ", process.memory_info().rss / (1000*1000*1000))

<br>

## Step 3. Hospital decrypts the COVID-19 predictions
Now, we are back in the trusted client environment where we can decrypt and decode the results. The client's side context has the private key so all objects on the client side can perform decryption.

In [ ]:
%%time
plain_predictions = iop.decrypt_decode_output(predictions)
process = psutil.Process(os.getpid())
print("Memory consumption (GB): ", process.memory_info().rss / (1000*1000*1000))

In [ ]:
print(f"\033[0;35m Classificaton prediction:",labels[np.argmax(plain_predictions)],"\033[0m")

<br>

#### References:

<sub><sup> 1. Kang Zhang, Xiaohong Liu, Jun Shen, et al. Jianxing He, Tianxin Lin, Weimin Li, Guangyu Wang. (2020). Clinically Applicable AI System for Accurate Diagnosis, Quantitative Measurements and Prognosis of COVID-19 Pneumonia Using Computed Tomography. Cell. DOI: https://doi.org/10.1016/j.cell.2020.04.045 <sub><sup> 

<sub><sup> 2. An P, Xu S, Harmon SA, Turkbey EB, Sanford TH, Amalou A, Kassin M, Varble N, Blain M, Anderson V, Patella F, Carrafiello G, Turkbey BT, Wood BJ (2020). CT Images in Covid-19 [Data set]. The Cancer Imaging Archive. DOI: https://doi.org/10.7937/tcia.2020.gqry-nc81 <sub><sup>

<sub><sup> 3. Ma Jun, Ge Cheng, Wang Yixin, An Xingle, Gao Jiantao, Yu Ziqi, … He Jian. (2020). COVID-19 CT Lung and Infection Segmentation Dataset (Version Verson 1.0) [Data set]. Zenodo. https://doi.org/10.5281/zenodo.3757476 <sub><sup>

<sub><sup> 4. Armato III, SG; McLennan, G; Bidaut, L; McNitt-Gray, MF; Meyer, CR; Reeves, AP; Zhao, B; Aberle, DR; Henschke, CI; Hoffman, Eric A; Kazerooni, EA; MacMahon, H; van Beek, EJR; Yankelevitz, D; Biancardi, AM; Bland, PH; Brown, MS; Engelmann, RM; Laderach, GE; Max, D; Pais, RC; Qing, DPY; Roberts, RY; Smith, AR; Starkey, A; Batra, P; Caligiuri, P; Farooqi, Ali; Gladish, GW; Jude, CM; Munden, RF; Petkovska, I; Quint, LE; Schwartz, LH; Sundaram, B; Dodd, LE; Fenimore, C; Gur, D; Petrick, N; Freymann, J; Kirby, J; Hughes, B; Casteele, AV; Gupte, S; Sallam, M; Heath, MD; Kuhn, MH; Dharaiya, E; Burns, R; Fryd, DS; Salganicoff, M; Anand, V; Shreter, U; Vastagh, S; Croft, BY; Clarke, LP. (2015). Data From LIDC-IDRI. The Cancer Imaging Archive. https://doi.org/10.7937/K9/TCIA.2015.LO9QL9SX <sub><sup>


<sub><sup> 5. An, P., Xu, S., Harmon, S. A., Turkbey, E. B., Sanford, T. H., Amalou, A., Kassin, M., Varble, N., Blain, M., Anderson, V., Patella, F., Carrafiello, G., Turkbey, B. T., & Wood, B. J. (2020). CT Images in COVID-19 [Data set]. The Cancer Imaging Archive. https://doi.org/10.7937/TCIA.2020.GQRY-NC815 <sub><sup>



<sub><sup> 6. Armato III, S. G., McLennan, G., Bidaut, L., McNitt-Gray, M. F., Meyer, C. R., Reeves, A. P., Zhao, B., Aberle, D. R., Henschke, C. I., Hoffman, E. A., Kazerooni, E. A., MacMahon, H., Van Beek, E. J. R., Yankelevitz, D., Biancardi, A. M., Bland, P. H., Brown, M. S., Engelmann, R. M., Laderach, G. E., Max, D., Pais, R. C. , Qing, D. P. Y. , Roberts, R. Y., Smith, A. R., Starkey, A., Batra, P., Caligiuri, P., Farooqi, A., Gladish, G. W., Jude, C. M., Munden, R. F., Petkovska, I., Quint, L. E., Schwartz, L. H., Sundaram, B., Dodd, L. E., Fenimore, C., Gur, D., Petrick, N., Freymann, J., Kirby, J., Hughes, B., Casteele, A. V., Gupte, S., Sallam, M., Heath, M. D., Kuhn, M. H., Dharaiya, E., Burns, R., Fryd, D. S., Salganicoff, M., Anand, V., Shreter, U., Vastagh, S., Croft, B. Y., Clarke, L. P. (2015). Data From LIDC-IDRI [Data set]. The Cancer Imaging Archive. https://doi.org/10.7937/K9/TCIA.2015.LO9QL9SX <sub><sup>

